# Подготовка данных

In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn import model_selection, metrics, tree, ensemble

In [2]:
# Получение датафрейма
df = pd.read_csv('../data/insurance_preprocessed.csv', index_col=0)

In [3]:
# Первые пять записей в датафрейме
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,1,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,3,21984.47061
4,32,1,28.880,0,0,3,3866.85520


In [4]:
# Информацию о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   int64  
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   int64  
 5   region    1338 non-null   int64  
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(5)
memory usage: 83.6 KB


In [5]:
# Количество нулевых значений
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [6]:
# Разделение признаков 
y = df['charges'].values
df.pop('charges')
X = df.values

In [7]:
# Разделение на обучаемую и тестовую выборки
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

In [8]:
# Масштабирование признаков
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Модель дерева решений

## Обучение модели дерева решений

In [9]:
# Создание модели
tree_reg = tree.DecisionTreeRegressor()

In [10]:
%%time
# Обучение модели
tree_reg.fit(X_train, y_train)

Wall time: 7 ms


DecisionTreeRegressor()

## Подбор гиперпараметров

In [11]:
# Текущие значения гиперпараметров модели
tree_reg.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'best'}

In [12]:
# подбираемые гиперпараметры
params = {
    'criterion' : ['mse', 'friedman_mse', 'mae'],
    'splitter' : ['best', 'random'],
    'max_depth': [1, 3, 5],
    'min_samples_split' : [2, 4, 6, 8, 10]
}

In [13]:
# Создание модели GridSearchCV
grid_tree_reg = model_selection.GridSearchCV(tree.DecisionTreeRegressor(), params)

In [14]:
%%time
# Обучение модели
grid_tree_reg.fit(X_train, y_train)

Wall time: 2.07 s


GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['mse', 'friedman_mse', 'mae'],
                         'max_depth': [1, 3, 5],
                         'min_samples_split': [2, 4, 6, 8, 10],
                         'splitter': ['best', 'random']})

In [108]:
# Оптимальная модель
grid_tree_reg.best_estimator_

DecisionTreeRegressor(criterion='mae', max_depth=5)

## Качество стандартной модели

In [16]:
y_pred = tree_reg.predict(X_test)

In [17]:
# R^2 score
metrics.r2_score(y_test, y_pred)

0.7052788167097463

In [18]:
# MSE score
metrics.mean_squared_error(y_test, y_pred)

44652858.19197827

In [19]:
# RMSE score
metrics.mean_squared_error(y_test, y_pred, squared=False)

6682.279415886339

In [20]:
# MAE score
metrics.mean_absolute_error(y_test, y_pred)

3275.089055597015

## Качество модели с гиперпараметрами

In [21]:
# Предсказанные значения
y_pred = grid_tree_reg.predict(X_test)

In [22]:
# R^2 score
metrics.r2_score(y_test, y_pred)

0.8208012947323277

In [23]:
# MSE score
metrics.mean_squared_error(y_test, y_pred)

27150184.066080656

In [24]:
# RMSE score
metrics.mean_squared_error(y_test, y_pred, squared=False)

5210.583850786844

In [25]:
# MAE score
metrics.mean_absolute_error(y_test, y_pred)

2358.0289882089555

# Обучение композиций

## Бэггинг

### Композиция без гиперпараметров

#### Обучение модели

In [26]:
# Создание модели
bag_reg = ensemble.BaggingRegressor(tree.DecisionTreeRegressor())

In [27]:
%%time
# Обучение модели
bag_reg.fit(X_train, y_train)

Wall time: 39 ms


BaggingRegressor(base_estimator=DecisionTreeRegressor())

#### Качество модели

In [28]:
# Предсказанные значения
y_pred = bag_reg.predict(X_test)

In [29]:
# R^2 score
metrics.r2_score(y_test, y_pred)

0.7958750524656603

In [30]:
# MSE score
metrics.mean_squared_error(y_test, y_pred)

30926729.575183876

In [31]:
# RMSE score
metrics.mean_squared_error(y_test, y_pred, squared=False)

5561.180591851327

In [32]:
# MAE score
metrics.mean_absolute_error(y_test, y_pred)

3087.3086785630594

Качество модели композиции без гиперпараметров выше, чем у стандартной модели дерева решений.

Качество модели композиции без гиперпараметров выше, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4 ms

Время работы (обучения) модели дерева решений с гиперпараметрами - 1.95 s

Время работы (обучения) композиции без гиперпараметров - 35.8 ms

### Композиция с гиперпараметрами у базовых алгоритмов

#### Обучение композиции

In [33]:
# Создание модели
bag_reg = ensemble.BaggingRegressor(tree.DecisionTreeRegressor(max_depth=3, min_samples_split=6))

In [34]:
%%time
# Обучение модели
bag_reg.fit(X_train, y_train)

Wall time: 24 ms


BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=3,
                                                      min_samples_split=6))

#### Качество модели

In [35]:
# Предсказанные значения
y_pred = bag_reg.predict(X_test)

In [36]:
# R^2 score
metrics.r2_score(y_test, y_pred)

0.8304645265076083

In [37]:
# MSE score
metrics.mean_squared_error(y_test, y_pred)

25686119.239383508

In [38]:
# RMSE score
metrics.mean_squared_error(y_test, y_pred, squared=False)

5068.147515550776

In [39]:
# MAE score
metrics.mean_absolute_error(y_test, y_pred)

2842.114129257444

Качество модели композиции с гиперпараметрами у базовых моделей выше, чем у стандартной модели дерева решений.

Качество модели композиции с гиперпараметрами у базовых моделей выше, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4 ms

Время работы (обучения) модели дерева решений с гиперпараметрами - 1.95 s

Время работы (обучения) композиции с гиперпараметрами у базовых моделей - 22.5 ms

### Композиция с гиперпараметрами

#### Обучение модели

In [40]:
# Создание модели
bag_reg = ensemble.BaggingRegressor(tree.DecisionTreeRegressor())

In [41]:
# Текущие значения гиперпараметров у модели
bag_reg.get_params()

{'base_estimator__ccp_alpha': 0.0,
 'base_estimator__criterion': 'mse',
 'base_estimator__max_depth': None,
 'base_estimator__max_features': None,
 'base_estimator__max_leaf_nodes': None,
 'base_estimator__min_impurity_decrease': 0.0,
 'base_estimator__min_impurity_split': None,
 'base_estimator__min_samples_leaf': 1,
 'base_estimator__min_samples_split': 2,
 'base_estimator__min_weight_fraction_leaf': 0.0,
 'base_estimator__presort': 'deprecated',
 'base_estimator__random_state': None,
 'base_estimator__splitter': 'best',
 'base_estimator': DecisionTreeRegressor(),
 'bootstrap': True,
 'bootstrap_features': False,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [42]:
# Подбираемые гиперпараметры
params = {
    'bootstrap' : [False, True],
    'bootstrap_features' : [False, True],
    'warm_start' : [False, True],
    'n_estimators' : [10, 15, 20, 25, 30, 40]
}

In [43]:
# Создание модели GridSearchCV
grid_bag_reg = model_selection.GridSearchCV(bag_reg, params)

In [44]:
%%time
# Обучение модели
grid_bag_reg.fit(X_train, y_train)

Wall time: 18.8 s


GridSearchCV(estimator=BaggingRegressor(base_estimator=DecisionTreeRegressor()),
             param_grid={'bootstrap': [False, True],
                         'bootstrap_features': [False, True],
                         'n_estimators': [10, 15, 20, 25, 30, 40],
                         'warm_start': [False, True]})

In [45]:
# Оптимальная модель
grid_bag_reg.best_estimator_

BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=40)

#### Качество модели

In [46]:
# Предсказанные значения
y_pred = grid_bag_reg.predict(X_test)

In [47]:
# R^2 score
metrics.r2_score(y_test, y_pred)

0.8042648650050376

In [48]:
# MSE score
metrics.mean_squared_error(y_test, y_pred)

29655598.99204847

In [49]:
# RMSE score
metrics.mean_squared_error(y_test, y_pred, squared=False)

5445.69545531592

In [50]:
# MAE score
metrics.mean_absolute_error(y_test, y_pred)

3060.744357345461

Качество модели композиции с гиперпараметрами выше, чем у стандартной модели дерева решений.

Качество модели композиции с гиперпараметрами выше, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4 ms

Время работы (обучения) модели дерева решений с гиперпараметрами - 1.95 s

Время работы (обучения) композиции с гиперпараметрами - 19.2 s

### Композиция с гиперпараметрами и с гиперпараметрами у базовых моделей

#### Обучение модели

In [51]:
# Создание модели
bag_reg = ensemble.BaggingRegressor(tree.DecisionTreeRegressor(max_depth=3, min_samples_split=6), 
                                    n_estimators=30,warm_start=True)

In [52]:
%%time
# Обучение модели
bag_reg.fit(X_train, y_train)

Wall time: 59 ms


BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=3,
                                                      min_samples_split=6),
                 n_estimators=30, warm_start=True)

#### Качество модели

In [53]:
# Предсказанные значения
y_pred = bag_reg.predict(X_test)

In [54]:
# R^2 score
metrics.r2_score(y_test, y_pred)

0.8257848071765108

In [55]:
# MSE score
metrics.mean_squared_error(y_test, y_pred)

26395137.98495484

In [56]:
# RMSE score
metrics.mean_squared_error(y_test, y_pred, squared=False)

5137.619875482696

In [57]:
# MAE score
metrics.mean_absolute_error(y_test, y_pred)

2857.821314188279

Качество модели композиции с гиперпараметрами и с гиперпараметрами у базовых моделей выше, чем у стандартной модели дерева решений.

Качество модели композиции с гиперпараметрами и с гиперпараметрами у базовых моделей выше, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4 ms

Время работы (обучения) модели дерева решений с гиперпараметрами - 1.95 s

Время работы (обучения) композиции с гиперпараметрами и с гиперпараметрами у базовых моделей - 62.5 ms

## Бустинг

### Композиция без гиперпараметров

#### Обучение модели

In [58]:
# Создание модели
grad_boost_reg = xgb.XGBRegressor()

In [59]:
%%time
# Обучение модели
grad_boost_reg.fit(X_train, y_train)

Wall time: 2.44 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

#### Качество модели

In [60]:
# Предсказанные значения
y_pred = grad_boost_reg.predict(X_test)

In [61]:
# R^2 score
metrics.r2_score(y_test, y_pred)

0.7745369396048216

In [62]:
# MSE score
metrics.mean_squared_error(y_test, y_pred)

34159641.838301

In [63]:
# RMSE score
metrics.mean_squared_error(y_test, y_pred, squared=False)

5844.625038298094

In [64]:
# MAE score
metrics.mean_absolute_error(y_test, y_pred)

3408.055715927976

Качество модели композиции без гиперпараметров выше, чем у стандартной модели дерева решений.

Качество модели композиции без гиперпараметров выше, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4 ms

Время работы (обучения) модели дерева решений с гиперпараметрами - 1.95 s

Время работы (обучения) композиции без гиперпараметров - 236 ms

### Композиция с гиперпараметрами

#### Обучение модели

In [65]:
# Создание модели
grad_boost_reg = xgb.XGBRegressor()

In [66]:
# Текущие значения гиперпараметров модели
grad_boost_reg.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'gamma': None,
 'gpu_id': None,
 'importance_type': 'gain',
 'interaction_constraints': None,
 'learning_rate': None,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [67]:
# Подбираемые гиперпараметры
params = {
    'n_estimators' : [100, 110, 120],
    'max_depth' : [3, 4, 5, 6, 7],
    'colsample_bytree' : [0.7, 0.8]
}

In [68]:
# Создание модели
grid_grad_boost_reg = model_selection.GridSearchCV(grad_boost_reg, params)

In [69]:
%%time
# Обучение модели
grid_grad_boost_reg.fit(X_train, y_train)

Wall time: 14.6 s


GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_parameters=None,
      

In [70]:
# Оптимальная модель
grid_grad_boost_reg.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

#### Оценка модели

In [71]:
# Предсказанные значения
y_pred = grid_grad_boost_reg.predict(X_test)

In [72]:
# R^2 score
metrics.r2_score(y_test, y_pred)

0.8105825541831673

In [73]:
# MSE score
metrics.mean_squared_error(y_test, y_pred)

28698413.370632876

In [74]:
# RMSE score
metrics.mean_squared_error(y_test, y_pred, squared=False)

5357.09000956983

In [75]:
# MAE score
metrics.mean_absolute_error(y_test, y_pred)

2988.66039297407

Качество модели композиции c гиперпараметрами выше, чем у стандартной модели дерева решений.

Качество модели композиции с гиперпарамерами выше, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4 ms

Время работы (обучения) модели дерева решений с гиперпараметрами - 1.95 s

Время работы (обучения) композиции с гиперпараметрами - 14.6 s

## Стекинг

### Композиция без гиперпараметров

#### Обучение модели

In [76]:
# Создание модели
stack_reg = ensemble.StackingRegressor([('dtr', tree.DecisionTreeRegressor())])

In [77]:
%%time
# Обучение модели
stack_reg.fit(X_train, y_train)

Wall time: 49 ms


StackingRegressor(estimators=[('dtr', DecisionTreeRegressor())])

#### Качество модели

In [78]:
# Предсказанные значения
y_pred = stack_reg.predict(X_test)

In [79]:
# R^2 score
metrics.r2_score(y_test, y_pred)

0.7155025322721495

In [80]:
# MSE score
metrics.mean_squared_error(y_test, y_pred)

43103875.12904886

In [81]:
# RMSE score
metrics.mean_squared_error(y_test, y_pred, squared=False)

6565.354151075848

In [82]:
# MAE score
metrics.mean_absolute_error(y_test, y_pred)

3712.5730849506763

Качество модели композиции без гиперпараметров выше, чем у стандартной модели дерева решений.

Качество модели композиции без гиперпараметров ниже, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4 ms

Время работы (обучения) модели дерева решений с гиперпараметрами - 1.95 s

Время работы (обучения) композиции без гиперпараметров - 30.1 ms

### Композиция с гиперпараметрами у базовых моделей

#### Обучение модели

In [83]:
# Создание модели
stack_reg = ensemble.StackingRegressor([('dtr', tree.DecisionTreeRegressor(max_depth=3, min_samples_split=6))])

In [84]:
%%time
# Обучение модели
stack_reg.fit(X_train, y_train)

Wall time: 17 ms


StackingRegressor(estimators=[('dtr',
                               DecisionTreeRegressor(max_depth=3,
                                                     min_samples_split=6))])

#### Качество модели

In [85]:
# Предсказанные значения
y_pred = stack_reg.predict(X_test)

In [86]:
# R^2 score
metrics.r2_score(y_test, y_pred)

0.8102909233957738

In [87]:
# MSE score
metrics.mean_squared_error(y_test, y_pred)

28742598.006594628

In [88]:
# RMSE score
metrics.mean_squared_error(y_test, y_pred, squared=False)

5361.212363504605

In [89]:
# MAE score
metrics.mean_absolute_error(y_test, y_pred)

3153.8326034948454

Качество модели композиции с гиперпараметрами у базовых моделей выше, чем у стандартной модели дерева решений.

Качество модели композиции с гиперпараметрами у базовых моделей ниже, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4 ms

Время работы (обучения) модели дерева решений с гиперпараметрами - 1.95 s

Время работы (обучения) композиции с гиперпараметрами у базовых моделей - 13 ms

### Композиция с гиперпараметрами

#### Обучение модели

In [90]:
# Создание модели
stack_reg = ensemble.StackingRegressor([('dtr', tree.DecisionTreeRegressor())])

In [91]:
# Текущие значения гиперпараметров модели
stack_reg.get_params()

{'cv': None,
 'estimators': [('dtr', DecisionTreeRegressor())],
 'final_estimator': None,
 'n_jobs': None,
 'passthrough': False,
 'verbose': 0,
 'dtr': DecisionTreeRegressor(),
 'dtr__ccp_alpha': 0.0,
 'dtr__criterion': 'mse',
 'dtr__max_depth': None,
 'dtr__max_features': None,
 'dtr__max_leaf_nodes': None,
 'dtr__min_impurity_decrease': 0.0,
 'dtr__min_impurity_split': None,
 'dtr__min_samples_leaf': 1,
 'dtr__min_samples_split': 2,
 'dtr__min_weight_fraction_leaf': 0.0,
 'dtr__presort': 'deprecated',
 'dtr__random_state': None,
 'dtr__splitter': 'best'}

In [92]:
# Подбираемые гиперпараметры
params = {
    'passthrough' : [False, True],
    'cv' : [5, 8, 10, 12]
}

In [93]:
# Создание модели GridSearchCV
grid_stack_reg = model_selection.GridSearchCV(stack_reg, params)

In [94]:
%%time
# Обучение модели
grid_stack_reg.fit(X_train, y_train)

Wall time: 1.63 s


GridSearchCV(estimator=StackingRegressor(estimators=[('dtr',
                                                      DecisionTreeRegressor())]),
             param_grid={'cv': [5, 8, 10, 12], 'passthrough': [False, True]})

In [95]:
# Оптимальная модель
grid_stack_reg.best_estimator_

StackingRegressor(cv=10, estimators=[('dtr', DecisionTreeRegressor())],
                  passthrough=True)

#### Качество модели

In [96]:
# Предсказанные значения
y_pred = grid_stack_reg.predict(X_test)

In [97]:
# R^2 score
metrics.r2_score(y_test, y_pred)

0.7693089854486863

In [98]:
# MSE score
metrics.mean_squared_error(y_test, y_pred)

34951722.99433441

In [99]:
# RMSE score
metrics.mean_squared_error(y_test, y_pred, squared=False)

5911.998223471858

In [100]:
# MAE score
metrics.mean_absolute_error(y_test, y_pred)

3652.562045411845

Качество модели композиции с гиперпараметрами выше, чем у стандартной модели дерева решений.

Качество модели композиции с гиперпараметрами ниже, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4 ms

Время работы (обучения) модели дерева решений с гиперпараметрами - 1.95 s

Время работы (обучения) композиции с гиперпараметрами - 1.34 s

### Композиция с гиперпараметрами и с гиперпараметрами у базовых моделей

#### Обучение модели

In [101]:
# Создание модели
stack_reg = ensemble.StackingRegressor([('dtr', tree.DecisionTreeRegressor(max_depth=3, min_samples_split=6))], 
                                       cv=10, passthrough=True)

In [102]:
%%time
# Обучение модели
stack_reg.fit(X_train, y_train)

Wall time: 30 ms


StackingRegressor(cv=10,
                  estimators=[('dtr',
                               DecisionTreeRegressor(max_depth=3,
                                                     min_samples_split=6))],
                  passthrough=True)

#### Качество модели

In [103]:
# Предсказанные значения
y_pred = stack_reg.predict(X_test)

In [104]:
# R^2 score
metrics.r2_score(y_test, y_pred)

0.8225065405973768

In [105]:
# MSE score
metrics.mean_squared_error(y_test, y_pred)

26891824.280250434

In [106]:
# RMSE score
metrics.mean_squared_error(y_test, y_pred, squared=False)

5185.732762132121

In [107]:
# MAE score
metrics.mean_absolute_error(y_test, y_pred)

3023.2526381455864

Качество модели композиции с гиперпараметрами и с гиперпараметрами у базовых моделей выше, чем у стандартной модели дерева решений.

Качество модели композиции с гиперпараметрами и с гиперпараметрами у базовых моделей выше, чем у модели дерева решений с гиперпараметрами.

Время работы (обучения) стандартной модели дерева решений - 4 ms

Время работы (обучения) модели дерева решений с гиперпараметрами - 1.95 s

Время работы (обучения) композиции с гиперпараметрами и с гиперпараметрами у базовых моделей - 18.2 ms